In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from os import getcwd, path
import urllib
import pickle
from time import sleep
import functools
import PyPDF2
import glob
from func_timeout import func_timeout, FunctionTimedOut
import re

In [3]:
PROJECT_NAME = 'modern_slavery'
PROJECT_PATH = f"{getcwd()[:getcwd().find(PROJECT_NAME)]}{PROJECT_NAME}"
DATA_PATH = f"{PROJECT_PATH}\\data"
PICKLE_PATH = f"{DATA_PATH}\\pickles"
PDF_PATH = f"{DATA_PATH}\\pdfs"
TEXT_PATH = f"{DATA_PATH}\\texts"
MAX_DOWNLOAD_TIME = 5

In [4]:
def save_pickle(obj, file_name):
    pickle.dump(obj, file=open(f"{file_name}.pickle","wb"))
    
def load_pickle(file_name):
    return pickle.load(file=open(f"{file_name}.pickle","rb"))

In [5]:
pickle_name = f"{PICKLE_PATH}\\extracted_text_SIDs"
if path.exists(f"{pickle_name}.pickle"):
    extracted_text_SIDs = load_pickle(pickle_name)
    print(f"{pickle_name} exists")
else:
    extracted_text_SIDs = []
    save_pickle(extracted_text_SIDs, file_name=pickle_name)
    print(f"{pickle_name} created")
    
    
pickle_name = f"{PICKLE_PATH}\\failed_text_SIDs"
if path.exists(f"{pickle_name}.pickle"):
    failed_text_SIDs = load_pickle(pickle_name)
    print(f"{pickle_name} exists")
else:
    failed_text_SIDs = []
    save_pickle(failed_text_SIDs, file_name=pickle_name)
    print(f"{pickle_name} created")

E:\RA Work\RA - Prof. Shawn Bhimani\modern_slavery\data\pickles\extracted_text_SIDs created
E:\RA Work\RA - Prof. Shawn Bhimani\modern_slavery\data\pickles\failed_text_SIDs created


In [6]:
df = pd.read_csv(f"{DATA_PATH}\modernslaveryregistry-2020-09-14.csv")
df.head()

,Company ID,Company,Is Publisher,Statement ID,URL,Override URL,Companies House Number,Industry,HQ,Is Also Covered,UK Modern Slavery Act,California Transparency in Supply Chains Act,Australia Modern Slavery Act,Period Covered
0,7676,"""K"" Line Holding Europe Limited",True,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,05005018,Marine,United Kingdom,False,True,False,False,2018-2019
1,28660,"""K"" Line Bulk Shipping (UK) Limited",False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,04830352,Marine,United Kingdom,True,True,False,False,2018-2019
2,28659,"""K"" Line (Europe) Limited",False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,05639474,Marine,United Kingdom,True,True,False,False,2018-2019
3,28661,"""K"" Line LNG Shipping Limited",False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,NaN,Marine,United Kingdom,True,True,False,False,2018-2019
4,28658,Polar LNG Shipping (UK) Limited,False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,02205323,Marine,United Kingdom,True,True,False,False,2018-2019


In [7]:
UNIQUE_SIDs = np.unique(df['Statement ID'].values)
TOTAL_UNIQUE_SIDs = len(UNIQUE_SIDs)
print(f"Total enteries : {len(df)}, "
      f"total unique statements : {TOTAL_UNIQUE_SIDs}")

Total enteries : 27531, total unique statements : 17799


In [8]:
pdfs = glob.glob(pathname=f"{PDF_PATH}\\*.pdf")
print(f"Found {len(pdfs)} pdfs.")

Found 11044 pdfs.


In [9]:
def get_sid(file_name):
    return int(re.findall(pattern=r"[\d]+",string=path.basename(file_name))[0])

In [10]:
def clean_text(text):
    return text.replace("\n", " ").strip()

In [11]:
def extract_text(pdf):
    pdf_file_obj = open(pdf, 'rb')
    pdf_reader = PyPDF2.PdfFileReader(pdf_file_obj)
    num_pages = pdf_reader.numPages
    text = ""
    for page in range(num_pages):
        page_obj = pdf_reader.getPage(page)
        text += page_obj.extractText()
    text =  clean_text(text)
    return " ".join(text.split())

In [12]:
def save_text(text, sid):
    file_name = f"SID-{sid}" 
    file_obj = open(f"{TEXT_PATH}//{file_name}.txt", 'w', encoding='utf-8')
    file_obj.write(text)
    file_obj.close()

In [13]:
for pdf in pdfs:
    sid = re.findall(pattern=r"[\d]+",string=path.basename(pdf))[0]
    print(f"Running {len(extracted_text_SIDs) + len(failed_text_SIDs)}/{len(pdfs)}")
    try:
        text = extract_text(pdf)
        save_text(text=text, sid=sid)
        extracted_text_SIDs.append(int(sid))
        save_pickle(extracted_text_SIDs, file_name=f"{PICKLE_PATH}\\extracted_text_SIDs")
        print(f"Extracted text for {len(extracted_text_SIDs)}/{len(pdfs)}")
    except:
        failed_text_SIDs.append(failed_text_SIDs)
        save_pickle(failed_text_SIDs, file_name=f"{PICKLE_PATH}\\failed_text_SIDs")
        print(f"Failed extract for {len(failed_text_SIDs)}/{len(pdfs)}")

Running 0/11044
Extracted text for 1/11044
Running 1/11044
Extracted text for 2/11044
Running 2/11044
Failed extract for 1/11044
Running 3/11044
Failed extract for 2/11044
Running 4/11044
Failed extract for 3/11044
Running 5/11044
Extracted text for 3/11044
Running 6/11044
Failed extract for 4/11044
Running 7/11044
Extracted text for 4/11044
Running 8/11044
Failed extract for 5/11044
Running 9/11044
Failed extract for 6/11044
Running 10/11044
Failed extract for 7/11044
Running 11/11044
Extracted text for 5/11044
Running 12/11044
Failed extract for 8/11044
Running 13/11044
Failed extract for 9/11044
Running 14/11044
Failed extract for 10/11044
Running 15/11044


Extracted text for 6/11044
Running 16/11044
Failed extract for 11/11044
Running 17/11044
Extracted text for 7/11044
Running 18/11044
Failed extract for 12/11044
Running 19/11044
Extracted text for 8/11044
Running 20/11044
Extracted text for 9/11044
Running 21/11044
Failed extract for 13/11044
Running 22/11044
Failed extract for 14/11044
Running 23/11044
Failed extract for 15/11044
Running 24/11044
Failed extract for 16/11044
Running 25/11044
Extracted text for 10/11044
Running 26/11044
Extracted text for 11/11044
Running 27/11044
Extracted text for 12/11044
Running 28/11044
Extracted text for 13/11044
Running 29/11044
Failed extract for 17/11044
Running 30/11044
Failed extract for 18/11044
Running 31/11044
Failed extract for 19/11044
Running 32/11044
Extracted text for 14/11044
Running 33/11044
Failed extract for 20/11044
Running 34/11044
Failed extract for 21/11044
Running 35/11044
Extracted text for 15/11044
Running 36/11044
Failed extract for 22/11044
Running 37/11044
Extracted text

Extracted text for 65/11044
Running 176/11044
Extracted text for 66/11044
Running 177/11044
Extracted text for 67/11044
Running 178/11044
Extracted text for 68/11044
Running 179/11044
Extracted text for 69/11044
Running 180/11044
Failed extract for 112/11044
Running 181/11044
Failed extract for 113/11044
Running 182/11044
Failed extract for 114/11044
Running 183/11044
Failed extract for 115/11044
Running 184/11044
Extracted text for 70/11044
Running 185/11044
Failed extract for 116/11044
Running 186/11044
Failed extract for 117/11044
Running 187/11044
Failed extract for 118/11044
Running 188/11044
Failed extract for 119/11044
Running 189/11044
Failed extract for 120/11044
Running 190/11044
Failed extract for 121/11044
Running 191/11044
Failed extract for 122/11044
Running 192/11044
Extracted text for 71/11044
Running 193/11044
Failed extract for 123/11044
Running 194/11044
Extracted text for 72/11044
Running 195/11044
Failed extract for 124/11044
Running 196/11044
Failed extract for 12

Extracted text for 162/11044
Running 403/11044
Extracted text for 163/11044
Running 404/11044
Failed extract for 242/11044
Running 405/11044
Failed extract for 243/11044
Running 406/11044
Failed extract for 244/11044
Running 407/11044
Failed extract for 245/11044
Running 408/11044
Failed extract for 246/11044
Running 409/11044
Failed extract for 247/11044
Running 410/11044
Failed extract for 248/11044
Running 411/11044
Failed extract for 249/11044
Running 412/11044
Failed extract for 250/11044
Running 413/11044
Extracted text for 164/11044
Running 414/11044
Extracted text for 165/11044
Running 415/11044
Failed extract for 251/11044
Running 416/11044
Failed extract for 252/11044
Running 417/11044
Failed extract for 253/11044
Running 418/11044
Failed extract for 254/11044
Running 419/11044
Extracted text for 166/11044
Running 420/11044
Failed extract for 255/11044
Running 421/11044
Extracted text for 167/11044
Running 422/11044
Extracted text for 168/11044
Running 423/11044
Failed extrac

Extracted text for 369/11044
Running 858/11044
Extracted text for 370/11044
Running 859/11044
Failed extract for 490/11044
Running 860/11044
Failed extract for 491/11044
Running 861/11044
Failed extract for 492/11044
Running 862/11044
Failed extract for 493/11044
Running 863/11044
Extracted text for 371/11044
Running 864/11044
Extracted text for 372/11044
Running 865/11044
Extracted text for 373/11044
Running 866/11044
Extracted text for 374/11044
Running 867/11044
Extracted text for 375/11044
Running 868/11044
Extracted text for 376/11044
Running 869/11044
Failed extract for 494/11044
Running 870/11044
Failed extract for 495/11044
Running 871/11044
Failed extract for 496/11044
Running 872/11044
Extracted text for 377/11044
Running 873/11044
Extracted text for 378/11044
Running 874/11044
Extracted text for 379/11044
Running 875/11044
Failed extract for 497/11044
Running 876/11044
Failed extract for 498/11044
Running 877/11044
Extracted text for 380/11044
Running 878/11044
Failed extrac

...

Extracted text for 3517/11044
Running 6975/11044
Extracted text for 3518/11044
Running 6976/11044
Failed extract for 3459/11044
Running 6977/11044
Failed extract for 3460/11044
Running 6978/11044
Extracted text for 3519/11044
Running 6979/11044
Failed extract for 3461/11044
Running 6980/11044
Failed extract for 3462/11044
Running 6981/11044
Failed extract for 3463/11044
Running 6982/11044
Failed extract for 3464/11044
Running 6983/11044
Failed extract for 3465/11044
Running 6984/11044
Failed extract for 3466/11044
Running 6985/11044
Extracted text for 3520/11044
Running 6986/11044
Extracted text for 3521/11044
Running 6987/11044
Extracted text for 3522/11044
Running 6988/11044
Extracted text for 3523/11044
Running 6989/11044
Extracted text for 3524/11044
Running 6990/11044
Failed extract for 3467/11044
Running 6991/11044
Failed extract for 3468/11044
Running 6992/11044
Extracted text for 3525/11044
Running 6993/11044
Extracted text for 3526/11044
Running 6994/11044
Failed extract for 3

Extracted text for 3553/11044
Running 7045/11044
Failed extract for 3493/11044
Running 7046/11044
Failed extract for 3494/11044
Running 7047/11044
Failed extract for 3495/11044
Running 7048/11044
Extracted text for 3554/11044
Running 7049/11044
Extracted text for 3555/11044
Running 7050/11044
Extracted text for 3556/11044
Running 7051/11044
Extracted text for 3557/11044
Running 7052/11044
Extracted text for 3558/11044
Running 7053/11044
Extracted text for 3559/11044
Running 7054/11044
Failed extract for 3496/11044
Running 7055/11044
Extracted text for 3560/11044
Running 7056/11044
Extracted text for 3561/11044
Running 7057/11044
Failed extract for 3497/11044
Running 7058/11044
Failed extract for 3498/11044
Running 7059/11044
Failed extract for 3499/11044
Running 7060/11044
Extracted text for 3562/11044
Running 7061/11044
Failed extract for 3500/11044
Running 7062/11044
Failed extract for 3501/11044
Running 7063/11044
Extracted text for 3563/11044
Running 7064/11044
Extracted text for 3

Extracted text for 3630/11044
Running 7181/11044
Failed extract for 3552/11044
Running 7182/11044
Failed extract for 3553/11044
Running 7183/11044
Extracted text for 3631/11044
Running 7184/11044
Extracted text for 3632/11044
Running 7185/11044
Failed extract for 3554/11044
Running 7186/11044
Extracted text for 3633/11044
Running 7187/11044
Extracted text for 3634/11044
Running 7188/11044
Extracted text for 3635/11044
Running 7189/11044
Extracted text for 3636/11044
Running 7190/11044
Extracted text for 3637/11044
Running 7191/11044
Failed extract for 3555/11044
Running 7192/11044
Failed extract for 3556/11044
Running 7193/11044
Extracted text for 3638/11044
Running 7194/11044
Extracted text for 3639/11044
Running 7195/11044
Extracted text for 3640/11044
Running 7196/11044
Extracted text for 3641/11044
Running 7197/11044
Failed extract for 3557/11044
Running 7198/11044
Extracted text for 3642/11044
Running 7199/11044
Failed extract for 3558/11044
Running 7200/11044
Extracted text for 3

Extracted text for 5029/11044
Running 9728/11044
Failed extract for 4700/11044
Running 9729/11044
Failed extract for 4701/11044
Running 9730/11044
Failed extract for 4702/11044
Running 9731/11044
Failed extract for 4703/11044
Running 9732/11044
Extracted text for 5030/11044
Running 9733/11044
Failed extract for 4704/11044
Running 9734/11044
Extracted text for 5031/11044
Running 9735/11044
Failed extract for 4705/11044
Running 9736/11044
Failed extract for 4706/11044
Running 9737/11044
Failed extract for 4707/11044
Running 9738/11044
Extracted text for 5032/11044
Running 9739/11044
Extracted text for 5033/11044
Running 9740/11044
Failed extract for 4708/11044
Running 9741/11044
Failed extract for 4709/11044
Running 9742/11044
Extracted text for 5034/11044
Running 9743/11044
Failed extract for 4710/11044
Running 9744/11044
Extracted text for 5035/11044
Running 9745/11044
Failed extract for 4711/11044
Running 9746/11044
Extracted text for 5036/11044
Running 9747/11044
Extracted text for 5

Extracted text for 5149/11044
Running 9918/11044
Extracted text for 5150/11044
Running 9919/11044
Extracted text for 5151/11044
Running 9920/11044
Failed extract for 4770/11044
Running 9921/11044
Failed extract for 4771/11044
Running 9922/11044
Failed extract for 4772/11044
Running 9923/11044
Extracted text for 5152/11044
Running 9924/11044
Failed extract for 4773/11044
Running 9925/11044
Extracted text for 5153/11044
Running 9926/11044
Extracted text for 5154/11044
Running 9927/11044
Failed extract for 4774/11044
Running 9928/11044
Extracted text for 5155/11044
Running 9929/11044
Failed extract for 4775/11044
Running 9930/11044
Failed extract for 4776/11044
Running 9931/11044
Extracted text for 5156/11044
Running 9932/11044
Extracted text for 5157/11044
Running 9933/11044
Failed extract for 4777/11044
Running 9934/11044
Extracted text for 5158/11044
Running 9935/11044
Failed extract for 4778/11044
Running 9936/11044
Failed extract for 4779/11044
Running 9937/11044
Extracted text for 5

In [ ]:
print(f"No. of pdf from which texts were extracted : {len(extracted_text_SIDs)}")
print(f"No. of pdf from which texts were not extracted : {len(failed_text_SIDs)}")

In [ ]:
len(np.unique([get_sid(file_name) for file_name in pdfs])), len(extracted_text_SIDs) + len(failed_text_SIDs)

In [ ]:
len(temp_sids)

In [ ]:
failed_text_SIDs = [i for i in failed_text_SIDs if i not in extracted_text_SIDs]

In [ ]:
saved_SIDs = list(np.unique([get_sid(file_name) for file_name in pdfs]))
pickle_name = f"{PICKLE_PATH}\\saved_SIDs"
save_pickle(saved_SIDs, file_name=pickle_name)